# Astro Lab Book 2025

In [1]:
import astropy
import ccdproc
from astropy.nddata import CCDData
from astropy.time.time_helper.function_helpers import linspace
from ccdproc import ccd_process
import numpy as np
from pathlib import Path
import ccdproc as ccdp
from astropy import units as u
import matplotlib.pyplot as plt
import photutils.background as bg
from astropy.convolution import convolve
from photutils.segmentation import make_2dgaussian_kernel
from photutils.segmentation import detect_sources
from photutils.segmentation import deblend_sources
from photutils.utils import calc_total_error
from photutils.segmentation import SourceCatalog
from photutils.aperture import CircularAperture, aperture_photometry
from photutils.background import MedianBackground
import os
import glob

### Combined Bias

In [2]:
raw_data = Path('data/bias')

calibrated_biases = glob.glob(str(raw_data / '*.fit*'))

combined_bias = ccdp.combine(calibrated_biases,
                             method='average',
                             mem_limit=350e6,
                             unit='adu'
                            )

combined_bias.meta['combined'] = True
combined_bias.write(raw_data / 'combined_bias.fit', overwrite=True)

INFO: splitting each image into 11 chunks to limit memory usage to 350000000.0 bytes. [ccdproc.combiner]


### Dark Frame Cleaning

In [5]:
path_dark_raw = 'data/darks'
combined_bias = astropy.nddata.CCDData.read('data/bias/combined_bias.fit', unit='adu')

for file_path in glob.glob(os.path.join(path_dark_raw,'*.fits')):
    ccd = ccdp.CCDData.read(file_path, unit='adu')
    ccd = ccdp.subtract_bias(ccd, combined_bias)
    file_name = os.path.basename(file_path)
    ccd.write('data/darks/bias_subtracted_' + file_name, overwrite = True)

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
